<a href="https://colab.research.google.com/github/emilyzfliu/vis-sounds/blob/main/vis_sounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import pickle
from math import pi
import torch
import imageio
from torch.autograd import Variable
from torch.utils.data import Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip /content/drive/MyDrive/vis_sound_files/vis-data-simple.zip
!cp /content/drive/MyDrive/vis_sound_files/train.txt .
!cp /content/drive/MyDrive/vis_sound_files/test.txt .

Archive:  /content/drive/MyDrive/vis_sound_files/vis-data-simple.zip
replace vis-data-simple/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Project Overview:

1. Visual sound classification
  * We will start with the ImageNet neural net architecture for training our model to predict labels from images
  * then labels from sounds (spectrogram images)
  * then finally combining sound and image data together.
2. Sound from images
  * First, use pretrained imagenet segmentation weights to identify components of image
  * Then, find association with sound annotation and find sounds closest to this (nearest neighbors?)

## Dataset
We use the Greatest Hits dataset at https://andrewowens.com/vis/.

In [ ]:
class VisSoundsDataset(Dataset):
  def __init__(self, split = 'train', mode='sound'):
    super(Dataset, self).__init__()
    assert split in ['train', 'test']
    self.split = split
    assert mode in ['sound', 'image', 'augment_sound', 'augment_image']
    self.mode = mode

    data, labels = self.load_data()

    self.data = {
        'inputs': data,
        'targets': labels
    }

    def load_vid_ids(self):
      with open(f'{self.split}.txt', 'r') as f:
        files = f.readlines()
      if self.mode == 'image':
        images = []
        labels = []
        for file in files:
          image = imageio.imread(f'vis-data-simple/{file}/{file}_image.png')
          img = np.moveaxis(img, 2, 0)
          img = np.array(img)
          images.append(img)
          labels.append(self.gen_labels(file))
        return images, labels
      elif self.mode == 'sound':
        images = []
        labels = []
        for file in files:
          with open(f'vis-data-simple/{file}/{file}_metadata.txt', 'r') as f:
            n_sounds = int(f.readline())
          for i in range(n_sounds):
            image = imageio.imread(f'vis-data-simple/{file}/{file}_spec_{i}.png')
            img = np.moveaxis(img, 2, 0)
            img = np.array(img)
            images.append(img)
            labels.append(self.gen_labels(file, i))
        return images, labels
      return None
    
    def gen_labels(file, idx=None):
      # TODO: FIGURE OUT MULTILABEL OUTPUTS
      ret = np.zeroes((2*17*5, 1))
      if idx is None:
        with open(f'vis-data-simple/{file}/{file}_metadata.txt', 'r') as f:
          n_sounds = int(f.readline())

    def __getitem__(self, item):
      return self.data['inputs'][item], self.data['targets'][item]
        
    def __len__(self):
        return self.n

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
# TODO: imagenet (resnet, vgg, etc) for pure sound/pure image classification